In [1]:
import numpy as np
import networkx as nx
import pickle as pk
import torch
import matplotlib.pyplot as plt
from stellargraph.data import UnsupervisedSampler
from tqdm import tqdm

from stellargraph import StellarGraph
from stellargraph.mapper import GraphSAGENodeGenerator,GraphSAGELinkGenerator
from stellargraph.data import EdgeSplitter
from stellargraph.layer import GraphSAGE, HinSAGE, link_classification

from tensorflow import keras
from sklearn import preprocessing, feature_extraction, model_selection

from stellargraph import globalvar
from stellargraph import datasets
from IPython.display import display, HTML

import logging
import time
from collections import defaultdict
import os
import scipy.sparse as sp
import torch
from torch.nn import functional as F

datasetname = 'SH_L-APIMethod'#有数据集SH_S、SH_L、MV,SH_L-APIMethod
threshold1 = 0.5
threshold2 = 0.5
batch_size = 1024
epochs = 100
num_samples = [20, 10]
layer_sizes = [64, 64]

best_suc = [0]*21
best_pre = [0]*21
best_recall = [0]*21
pro_best_suc = [0]*21
pro_best_pre = [0]*21
pro_best_recall = [0]*21
test_config = 'C2.1'

def getg(data):
    g = nx.Graph()
    n = len(data.item_method_id)
    # 顶点
    point = []
    for i in range(n):
        point.append(i)
    g.add_nodes_from(point)
    # 边权重
    edglist = []
    edges = set()
    for user, items in tqdm(data.invocation_mx.items()):
        for i in range(len(items)):
            for j in range(i+1,len(items)):
                edges.add((items[i],items[j]))
    
    for edg in tqdm(edges):
        edglist.append((edg[0],edg[1],float(data.adj[edg[0],edg[1]])))
        #edglist.append((edg[0],edg[1]))

    g.add_weighted_edges_from(edglist)
    #g.add_edges_from(edglist)
    return g

def to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

def build_my_new_adj_matrix(data,train_dict):
    n = len(data.item_method_id)
    A = sp.dok_matrix((n, n), dtype=np.float32)
    FD = defaultdict(int)
    for user, items in tqdm(train_dict.items()):
        for i in range(len(items)):
            FD[items[i]] += 1
            for j in range(i+1,len(items)):
                if A[items[i],items[j]] == 0:
                    A[items[i],items[j]] = A[items[j],items[i]] = len(data.invocation_mx.items())
                else:
                    A[items[i], items[j]] = A[items[j], items[i]] = A[items[j], items[i]] + len(data.invocation_mx.items())
    print('build_my_new_adj_matrix finish')
    data.FD = FD
    data.adj = to_torch_sparse_tensor(A)
    

def load_mydata(dataset_name):
    name = './tmp/%s-mydata.pk' % dataset_name
    if not os.path.exists(name):
        print('no file.')
    with open(name, 'rb') as f:
        data = pk.load(f)
        print('load dataset from disk.')
    #data.adj = to_torch_sparse_tensor(data.adj)
    return data

def get_node_embeddings(data):
    g = getg(data)
    g_feature_attr = g.copy()
    for node_id, node_data in g_feature_attr.nodes(data=True):
        node_data["feature"] = data.item_pre_emb[node_id].numpy()

    G = StellarGraph.from_networkx(
        g_feature_attr, node_features="feature", node_type_default="API", edge_type_default="Attribute"
    )
    edge_splitter_test = EdgeSplitter(G)

    # Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G, and obtain the
    # reduced graph G_test with the sampled links removed:
    G_test, edge_ids_test, edge_labels_test = edge_splitter_test.train_test_split(
        p=0.1, method="global", keep_connected=True
    )
    # Define an edge splitter on the reduced graph G_test:
    edge_splitter_train = EdgeSplitter(G_test)

    # Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G_test, and obtain the
    # reduced graph G_train with the sampled links removed:
    G_train, edge_ids_train, edge_labels_train = edge_splitter_train.train_test_split(
        p=0.1, method="global", keep_connected=True
    )

    train_gen = GraphSAGELinkGenerator(G_train, batch_size, num_samples)
    train_flow = train_gen.flow(edge_ids_train, edge_labels_train, shuffle=True)
    test_gen = GraphSAGELinkGenerator(G_test, batch_size, num_samples)
    test_flow = test_gen.flow(edge_ids_test, edge_labels_test)
    graphsage = GraphSAGE(
        layer_sizes=layer_sizes, generator=train_gen, bias=True, dropout=0.3
    )
    x_inp, x_out = graphsage.in_out_tensors()
    prediction = link_classification(
        output_dim=1, output_act="relu", edge_embedding_method="ip"
    )(x_out)
    model = keras.Model(inputs=x_inp, outputs=prediction)

    model.compile(
        optimizer=keras.optimizers.Adam(lr=1e-3),
        loss=keras.losses.binary_crossentropy,
        metrics=[keras.metrics.binary_accuracy],
    )
    history = model.fit(train_flow, epochs=epochs, validation_data=test_flow, verbose=2)
    x_inp_src = x_inp[0::2]
    x_out_src = x_out[0]
    embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)
    node_ids = G_train.nodes()
    node_gen = GraphSAGENodeGenerator(G_train, batch_size, num_samples).flow(node_ids)
    node_embeddings = embedding_model.predict(node_gen, workers=4, verbose=1)
    return node_embeddings

#threshold1取值（0，1）表示考虑节点相似特征的阈值，值越大候选特征节点越少
#threshold2取值（0，1）表示节点属性特征的重要性，越小越不重要
def build_new_relation(ratings,threshold1 = 0.8,threshold2 = 0.001):
    dataset = load_mydata(datasetname)
    n = len(ratings)
    for i in tqdm(range(n)):
        for j in range(i+1,n):
            simily = float(ratings[i][j])
            if simily >= threshold1:
                dataset.adj[i,j] = threshold2*simily
    return dataset
    
def get_my_top_items(tensor):
    item_dict = {}
    for i in tqdm(range(len(tensor))):
        if tensor[i].item() !=0 :
            item_dict[i] = tensor[i].item()
    #print('get_my_top_items==>item_dict',item_dict)
    top_items = [item[0] for item in sorted(item_dict.items(),key=lambda item:item[1],reverse=True)]
    #print('get_my_top_items==>top_items',top_items)
    return top_items[:21]

def get_my_top_items2(data,adj,Q,ratings,a,b):
    #链路预测
    diag = torch.diag(ratings) #获取对角为一维向量
    diag_embed = torch.diag_embed(diag)  # 由diag恢复为对角矩阵
    link_embed = ratings - diag_embed
    rowsoftmax = F.softmax(link_embed,dim=1)
    link_q1 = torch.zeros(size=[len(rowsoftmax[0])])
    for q in Q:
        link_q1 = link_q1 + rowsoftmax[q]
        
    #贝叶斯预测
    M = len(data.invocation_mx.items())
    D = set()
    FD = data.FD
    
    for q in Q:
        tensor = adj[q]
        for i in range(len(tensor)):
            if tensor[i].item() != 0:
                D.add(i)

    link_q2 = torch.zeros(size=[len(adj)])
    print(len(link_q2))
    for d in D:
        fd = FD[d]
        fdq = 1
        for q in Q:
            tensor = adj[q]
            fdq = fdq*(tensor[d].item()*1.0/fd)
        #利用贝叶斯求得d被预测的概率
        p2 = fdq*fd*1.0/M
        link_q2[d] = p2
    

    link_q = link_q1*a+link_q2*b
    arr,top_items = torch.sort(link_q,descending=True)
    top_items = top_items[:21]
    #top_items = [item[0] for item in sorted(item_dict.items(), key=lambda item: item[1], reverse=True)]
    # print('get_my_top_items==>top_items',top_items)
    return top_items.numpy()

def myeval(dataset,ratings,a,b):
    test_set = dataset.test_dict
    logger.info('test start. test set size: %d' % len(test_set))
    t1 = time.time()
    users = np.asarray(list(test_set.keys()))  # 训练集的方法编号数组

    top_items = []
    used_items = []

    for userid in tqdm(users):
        used_items.append(set(dataset.train_dict[userid]))
        #print(dataset.train_dict[userid],dataset.train_dict[userid][0])
        #top_items.append(get_my_top_items(dataset.adj[dataset.train_dict[userid][0]]))
        top_items.append(get_my_top_items2(dataset,dataset.adj,dataset.train_dict[userid],ratings,a,b))

    #print('myeval2=>top_items',top_items)
    #print('myeval2=>used_items', used_items)

    items = []
    for i, item in enumerate(top_items):  # 第i个测试方法推荐的API列表item
        # if i<=20:
        rec_item = [tid for tid in item if tid not in used_items[i]]
        # print(rec_item)
        items.append(rec_item[:20])

    def getMAP(N):
        qarr = []
        for i, uid in enumerate(users):
            r = 0
            drarr = []
            for k in range(1, N+1):
                intersect = set(items[i][:k]) & set(test_set[uid])
                p = len(intersect) / k
                newr = len(intersect) / len(set(test_set[uid]))
                dr = (newr-r)*p
                drarr.append(dr)
                r = newr
            qarr.append(np.sum(drarr))
        return np.sum(qarr)/len(qarr)
     
    def res_at_k(k):
        suc_methods = []
        precisions = []
        recalls = []
        proj_suc = defaultdict(list)
        proj_pre = defaultdict(list)
        proj_recall = defaultdict(list)

        for i, uid in enumerate(users):
            pid = dataset.test_user2proj[uid]
            intersect = set(items[i][:k]) & set(test_set[uid])
            if len(intersect) > 0:
                suc_methods.append(uid)
                proj_suc[pid].append(1)
            else:
                logger.debug('failed uid %d' % uid)
                logger.debug('GT:{}, REC:{}'.format(test_set[uid], items[i]))
                proj_suc[pid].append(0)
            p = len(intersect) / k
            r = len(intersect) / len(set(test_set[uid]))
            precisions.append(p)
            recalls.append(r)
            proj_pre[pid].append(p)
            proj_recall[pid].append(r)
        suc_rate = len(suc_methods) / len(users)
        logger.info('----------------------result@%d--------------------------' % k)
        logger.info('success rate at method level %f' % (suc_rate))
        logger.info('mean precision:%f, mean recall:%f' % (np.mean(precisions), np.mean(recalls)))

        suc_project = [np.mean(val) for val in proj_suc.values()]
        pres = [np.mean(val) for val in proj_pre.values()]
        recs = [np.mean(val) for val in proj_recall.values()]
        logger.info('**********************************************************')
        logger.info('success rate at project level %f' % (np.mean(np.mean(suc_project))))
        logger.info('mean precision:%f, mean recall:%f' % (np.mean(pres), np.mean(recs)))
        return suc_rate, np.mean(precisions), np.mean(recalls),np.mean(np.mean(suc_project)),np.mean(pres), np.mean(recs)

    t2 = time.time()
    logger.info('test end time: {}s'.format(t2 - t1))
    for i in range(1, 21):
        suc, pre, rec, pro_suc, pro_pre, pro_rec = res_at_k(i)
        if suc > best_suc[i]:
            best_suc[i] = suc
        if pre > best_pre[i]:
            best_pre[i] = pre
        if rec > best_recall[i]:
            best_recall[i] = rec
        logger.warning('method level => top %d : best suc %f, best pre %f,  best recall %f' % (i, best_suc[i], best_pre[i], best_recall[i]))

        if pro_suc > pro_best_suc[i]:
            pro_best_suc[i] = pro_suc
        if pro_pre > pro_best_pre[i]:
            pro_best_pre[i] = pro_pre
        if pro_rec > pro_best_recall[i]:
            pro_best_recall[i] = pro_rec
        logger.warning('project level => top %d : best suc %f, best pre %f,  best recall %f' % (i, pro_best_suc[i], pro_best_pre[i], pro_best_recall[i]))

    logger.info('MAP: %f' % (getMAP(20)))

In [2]:
#载入数据并划分数据集
data = load_mydata(datasetname)
data.split_data(test_config)
build_my_new_adj_matrix(data,data.train_dict)

load dataset from disk.
total user methods:189026, test_proj:{1, 513, 6, 8, 10, 12, 14, 15, 17, 531, 532, 21, 534, 31, 546, 547, 34, 549, 37, 552, 45, 558, 48, 50, 52, 55, 568, 62, 64, 577, 66, 581, 583, 75, 588, 78, 85, 603, 92, 608, 103, 122, 127, 142, 157, 165, 170, 172, 178, 181, 187, 190, 193, 194, 196, 202, 205, 206, 210, 211, 222, 225, 229, 233, 239, 242, 243, 245, 246, 249, 250, 271, 279, 293, 294, 304, 312, 318, 319, 320, 325, 339, 346, 350, 351, 358, 362, 366, 372, 378, 379, 382, 385, 389, 390, 392, 395, 399, 401, 409, 419, 425, 432, 433, 436, 440, 441, 443, 446, 449, 455, 456, 465, 468, 470, 482, 485, 487, 492, 493, 504, 506}
test set methods count:6844, invocations:44970


 13%|█▎        | 25403/189026 [00:00<00:00, 253991.49it/s]

load train datas ...


  0%|          | 196/189026 [00:00<01:37, 1936.06it/s]

train set methods count:189026, invocation: 3212745


100%|██████████| 189026/189026 [04:48<00:00, 654.89it/s] 


build_my_new_adj_matrix finish


In [3]:
node_embeddings = get_node_embeddings(data)

100%|██████████| 774984/774984 [4:26:30<00:00, 48.47it/s]   


** Sampled 68004 positive and 68004 negative edges. **
** Sampled 61204 positive and 61204 negative edges. **
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: [Errno 28] No space left on device: '/tmp/tmpu8_sd3mi.py'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: [Errno 28] No space left on device: '/tmp/tmpu8_sd3mi.py'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
link_classification: using 'ip' method to combine node embeddings into edge embeddings
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause

/root/anaconda3/lib/python3.6/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: [Errno 28] No space left on device: '/tmp/tmpti264iij.py'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: [Errno 28] No space left on device: '/tmp/tmpti264iij.py'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: [Errno 28] No space left on device: '/tmp/tmpibdr12f4.py'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the Tenso

Epoch 55/100
120/120 - 648s - loss: 0.6292 - binary_accuracy: 0.6474 - val_loss: 0.7401 - val_binary_accuracy: 0.6211
Epoch 56/100
120/120 - 613s - loss: 0.6159 - binary_accuracy: 0.6603 - val_loss: 0.6784 - val_binary_accuracy: 0.6539
Epoch 57/100
120/120 - 623s - loss: 0.6054 - binary_accuracy: 0.6703 - val_loss: 0.6624 - val_binary_accuracy: 0.6619
Epoch 58/100
120/120 - 622s - loss: 0.6007 - binary_accuracy: 0.6775 - val_loss: 0.6707 - val_binary_accuracy: 0.6613
Epoch 59/100
120/120 - 626s - loss: 0.5921 - binary_accuracy: 0.6835 - val_loss: 0.6512 - val_binary_accuracy: 0.6718
Epoch 60/100
120/120 - 651s - loss: 0.5939 - binary_accuracy: 0.6860 - val_loss: 0.6807 - val_binary_accuracy: 0.6571
Epoch 61/100
120/120 - 611s - loss: 0.6383 - binary_accuracy: 0.6667 - val_loss: 0.9381 - val_binary_accuracy: 0.5457
Epoch 62/100
120/120 - 617s - loss: 0.6139 - binary_accuracy: 0.6711 - val_loss: 1.2960 - val_binary_accuracy: 0.5171
Epoch 63/100
120/120 - 615s - loss: 1.1174 - binary_accu

In [4]:
#得到节点嵌入和链接相似性
api_embeddings = torch.from_numpy(node_embeddings)
ratings = api_embeddings.mm(api_embeddings.transpose(0, 1))

In [5]:
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time())) 

logging.basicConfig(format='%(asctime)s-%(levelname)s:%(message)s',
                    filename='./log/GLAPI-GraphSAGE-noPro_'+datasetname+'_'+str(threshold1)+'_'+str(threshold2)+'_'+str(batch_size)+'_'+str(epochs)+'_'+now+'.log',
                    filemode='a', level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
t1 = time.time()
myeval(data,ratings,threshold1,threshold2)
t2 = time.time()
logger.warning('predition time : %d' % (t2-t1))

  0%|          | 0/6844 [00:00<?, ?it/s]

30576


  0%|          | 1/6844 [00:07<14:45:25,  7.76s/it]

30576


  0%|          | 2/6844 [00:10<9:39:50,  5.08s/it] 

...

30576


 93%|█████████▎| 6344/6844 [95:20:12<1:20:10,  9.62s/it]

30576


 93%|█████████▎| 6345/6844 [95:20:25<1:28:56, 10.69s/it]

30576
